# Рекомендация тарифов

### Друзык Роман Богданович

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откроем и изучим файл

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import matplotlib.pyplot as plt

Загрузим датасет и посмотри на него

In [3]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
data.shape

(3214, 5)

In [5]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Мы видим, что наш набор данных состоит из 3214 событий и 5 признаков, один из которых является целевым (is_ultra).
Признак is_ultra является категориальным, поэтому перед нами стоит задача классификации. Наша модель должна предсказывать 1, если мы хотим порекомендовать клиенту тариф "Ultra" и 0, если будем рекомендовать тариф "Smart"

## 2. Разбъем данные на обучающую, валидационную и тестовую выборки

Так как у нас отсутвует тестовый набор данных, то правильней будет разбить выборку на три части: обучающая, валидационная и тестовая. На обучающей и валидационной выборках будем тренировать и проверять правильность наших моделей, на тестовой выборке проведем финальное тестирование нашей модели.

Сначала отдельно выделим признаки и решение

In [6]:
X = data.drop(['is_ultra'], axis=1)
y = data['is_ultra']

Разобъем на обучающую и валидационную, в пропорции 60/40.

Мой коммент: сделаю как ты сказал

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=.8, random_state=12)

Теперь валидационную выборку разобъем пополам, на валидационную и тестовую

In [8]:
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, train_size=.5, random_state=8)

В итоге мы разбили наш набор данных в пропорции 80/10/10. Приступим к выбору модели

## 3. Исследуем различные модели

На данном этапе мы займемся подбором модели классификации. Наш выбор будет состоять из следующих вариантов:
 * Метод k ближайших соседей
 * Логистическая регрессия
 * Классификатор линейных векторов
 * Дерево решений
 * Случайный лес
 * Градиентный бустинг деревьев решений
 * Простейшая нейронная сеть

***

#### Метод k ближайших соседей

В данном методе важный гиперпараметр - количество соседей, проанализируем с количестовм от 1 до 

In [9]:
from sklearn.neighbors import KNeighborsClassifier

In [11]:
for n_neighbors in range(1, 15):
    knc = KNeighborsClassifier(n_neighbors=n_neighbors)
    pred_k = knc.fit(X_train, y_train).predict(X_valid)
    print(f'n: {n_neighbors}\tточность на обучающем наборе: {round(knc.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_k),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_k), 3)}')

n: 1	точность на обучающем наборе: 1.0    	accuracy: 0.726    	ROC AUC: 0.662
n: 2	точность на обучающем наборе: 0.844    	accuracy: 0.741    	ROC AUC: 0.629
n: 3	точность на обучающем наборе: 0.858    	accuracy: 0.741    	ROC AUC: 0.661
n: 4	точность на обучающем наборе: 0.816    	accuracy: 0.748    	ROC AUC: 0.636
n: 5	точность на обучающем наборе: 0.823    	accuracy: 0.738    	ROC AUC: 0.649
n: 6	точность на обучающем наборе: 0.809    	accuracy: 0.741    	ROC AUC: 0.636
n: 7	точность на обучающем наборе: 0.812    	accuracy: 0.751    	ROC AUC: 0.655
n: 8	точность на обучающем наборе: 0.805    	accuracy: 0.748    	ROC AUC: 0.636
n: 9	точность на обучающем наборе: 0.806    	accuracy: 0.763    	ROC AUC: 0.662
n: 10	точность на обучающем наборе: 0.796    	accuracy: 0.751    	ROC AUC: 0.643
n: 11	точность на обучающем наборе: 0.801    	accuracy: 0.754    	ROC AUC: 0.648
n: 12	точность на обучающем наборе: 0.796    	accuracy: 0.751    	ROC AUC: 0.638
n: 13	точность на обучающем наборе: 0.7

Самый высокий параметр accuracy при n=9

***

#### Логистическая регрессия

В данном методе выберем гиперпараметры penalty (l1, l2) и solver (liblinear, lbfgs)

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
#Выберем penalty='l1', solver='liblinear', max_iter=150, чтобы не возникала ошибка
logreg = LogisticRegression(penalty='l1', 
                            solver='liblinear', 
                            max_iter=150)

pred_logreg = logreg.fit(X_train, y_train).predict(X_valid)

print(f'точность на обучающем наборе: {round(logreg.score(X_train, y_train), 3)}\
\taccuracy: {round(accuracy_score(y_valid.values, pred_logreg),3)}\
\tROC AUC: {round(roc_auc_score(y_valid.values, pred_logreg), 3)}')

точность на обучающем наборе: 0.759	accuracy: 0.738	ROC AUC: 0.599


In [14]:
#Выберем penalty='l2', solver='lbfgs', max_iter=150, чтобы не возникала ошибка
logreg = LogisticRegression(penalty='l2', 
                            solver='lbfgs', 
                            max_iter=150)

pred_logreg = logreg.fit(X_train, y_train).predict(X_valid)

print(f'точность на обучающем наборе: {round(logreg.score(X_train, y_train), 3)}\
\taccuracy: {round(accuracy_score(y_valid.values, pred_logreg),3)}\
\tROC AUC: {round(roc_auc_score(y_valid.values, pred_logreg), 3)}')

точность на обучающем наборе: 0.713	accuracy: 0.685	ROC AUC: 0.517


При выборе в логистической регрессии алгоритма решающего ядра параметры accuracy и ROC AUC меняются незначительно.

***

#### Классификатор линейных векторов

In [15]:
from sklearn.svm import LinearSVC

In [16]:
lsvc = LinearSVC(max_iter=100, 
                 random_state=7)

pred_lsvc = lsvc.fit(X_train, y_train).predict(X_valid)

print(f'точность на обучающем наборе: {round(lsvc.score(X_train, y_train), 3)}\
\taccuracy: {round(accuracy_score(y_valid.values, pred_lsvc),3)}\
\tROC AUC: {round(roc_auc_score(y_valid.values, pred_lsvc), 3)}')

точность на обучающем наборе: 0.726	accuracy: 0.701	ROC AUC: 0.543


/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Классификатор опорных векторов на данном наборе данных не показывает достаточной точности, даже на обучающем наборе.

***

#### Дерево решений

In [17]:
from sklearn.tree import DecisionTreeClassifier

Посмотрим как глубина решающего дерева влияет на точность и ROC AUC

In [18]:
for max_depth in range(5, 20):
    dtc = DecisionTreeClassifier(max_depth=max_depth, 
                                 random_state=2)
    
    pred_dtc = dtc.fit(X_train, y_train).predict(X_valid)
    
    print(f'Глубина: {max_depth}\
    \tточность на обучающем наборе: {round(dtc.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_dtc),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_dtc), 3)}')

Глубина: 5    	точность на обучающем наборе: 0.825    	accuracy: 0.776    	ROC AUC: 0.679
Глубина: 6    	точность на обучающем наборе: 0.839    	accuracy: 0.769    	ROC AUC: 0.672
Глубина: 7    	точность на обучающем наборе: 0.85    	accuracy: 0.782    	ROC AUC: 0.686
Глубина: 8    	точность на обучающем наборе: 0.859    	accuracy: 0.782    	ROC AUC: 0.699
Глубина: 9    	точность на обучающем наборе: 0.872    	accuracy: 0.782    	ROC AUC: 0.696
Глубина: 10    	точность на обучающем наборе: 0.88    	accuracy: 0.769    	ROC AUC: 0.687
Глубина: 11    	точность на обучающем наборе: 0.893    	accuracy: 0.763    	ROC AUC: 0.682
Глубина: 12    	точность на обучающем наборе: 0.903    	accuracy: 0.776    	ROC AUC: 0.699
Глубина: 13    	точность на обучающем наборе: 0.916    	accuracy: 0.745    	ROC AUC: 0.678
Глубина: 14    	точность на обучающем наборе: 0.93    	accuracy: 0.741    	ROC AUC: 0.691
Глубина: 15    	точность на обучающем наборе: 0.942    	accuracy: 0.741    	ROC AUC: 0.679
Глубина

Видим, что максимальное значение точности и ROC AUC у нас при глубине дерева - 13. Запомним это значение и будем использовать при дальнейших расчетах.

Далее проверим, как минимальное количество листов влияет на точность решающего дерева, при фиксированной глубине в 12

In [31]:
for min_samples_leaf  in range(10, 50, 2):
    dtc = DecisionTreeClassifier(max_depth=13, 
                                 min_samples_leaf=min_samples_leaf, 
                                 random_state=2)
    
    pred_dtc = dtc.fit(X_train, y_train).predict(X_valid)
    
    print(f'min число в листе: {min_samples_leaf}\
    \tточность на обучающем наборе: {round(dtc.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_dtc),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_dtc), 3)}')

min число в листе: 10    	точность на обучающем наборе: 0.852    	accuracy: 0.782    	ROC AUC: 0.719
min число в листе: 12    	точность на обучающем наборе: 0.847    	accuracy: 0.779    	ROC AUC: 0.714
min число в листе: 14    	точность на обучающем наборе: 0.842    	accuracy: 0.779    	ROC AUC: 0.716
min число в листе: 16    	точность на обучающем наборе: 0.837    	accuracy: 0.782    	ROC AUC: 0.701
min число в листе: 18    	точность на обучающем наборе: 0.836    	accuracy: 0.782    	ROC AUC: 0.704
min число в листе: 20    	точность на обучающем наборе: 0.835    	accuracy: 0.782    	ROC AUC: 0.699
min число в листе: 22    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число в листе: 24    	точность на обучающем наборе: 0.83    	accuracy: 0.785    	ROC AUC: 0.701
min число в листе: 26    	точность на обучающем наборе: 0.829    	accuracy: 0.785    	ROC AUC: 0.701
min число в листе: 28    	точность на обучающем наборе: 0.826    	accuracy: 0.785    	ROC AUC

Видим, что максимальное значение точности и ROC AUC у нас при глубине дерева - 13, минимальном числе объектов в листе - 22. Запомним это значение и будем использовать при дальнейших расчетах.

In [20]:
for min_samples_split  in range(5, 100, 5):
    dtc = DecisionTreeClassifier(max_depth=12, 
                                 min_samples_leaf=22, 
                                 min_samples_split=min_samples_split, 
                                 random_state=2)
    
    pred_dtc = dtc.fit(X_train, y_train).predict(X_valid)
    
    print(f'min число примеров: {min_samples_split}\
    \tточность на обучающем наборе: {round(dtc.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_dtc),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_dtc), 3)}')

min число примеров: 5    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число примеров: 10    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число примеров: 15    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число примеров: 20    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число примеров: 25    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число примеров: 30    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число примеров: 35    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число примеров: 40    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число примеров: 45    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72
min число примеров: 50    	точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC A

Видим что значительных изменений точности и ROC AUC мы не наблюдаем. Зафикисируем min_samples_split=5

Итоговое дерево:

In [21]:
dtc = DecisionTreeClassifier(max_depth=12, 
                             min_samples_leaf=22, 
                             min_samples_split=5, 
                             random_state=2)

pred_dtc = dtc.fit(X_train, y_train).predict(X_valid)

print(f'точность на обучающем наборе: {round(dtc.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_dtc),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_dtc), 3)}')

точность на обучающем наборе: 0.834    	accuracy: 0.801    	ROC AUC: 0.72


Видим что решающее дерево на валидационной выборке удовлетворяет нашим минимальным требованиям в уровне точности в 0.75. Запомним параметры этого дерева и будем его использовать при построении случайного леса.

***

#### Случайный лес деревьев решений

In [22]:
from sklearn.ensemble import RandomForestClassifier

При построении случайного леса используем параметры дерева, которые получили ранее. Выберем размер леса, перебирая количество деревьев.

In [23]:
for n_estimators in range(10, 200, 10):
    
    rfc = RandomForestClassifier(n_estimators=n_estimators, 
                                 max_depth=12, 
                                 random_state=9)
    
    pred_rfc = rfc.fit(X_train, y_train).predict(X_valid)
    
    print(f'Деревьев: {n_estimators}\
    \tточность на обучающем наборе: {round(rfc.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_rfc),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_rfc), 3)}')

Деревьев: 10    	точность на обучающем наборе: 0.907    	accuracy: 0.766    	ROC AUC: 0.684
Деревьев: 20    	точность на обучающем наборе: 0.907    	accuracy: 0.785    	ROC AUC: 0.708
Деревьев: 30    	точность на обучающем наборе: 0.904    	accuracy: 0.788    	ROC AUC: 0.713
Деревьев: 40    	точность на обучающем наборе: 0.905    	accuracy: 0.794    	ROC AUC: 0.718
Деревьев: 50    	точность на обучающем наборе: 0.905    	accuracy: 0.791    	ROC AUC: 0.715
Деревьев: 60    	точность на обучающем наборе: 0.905    	accuracy: 0.791    	ROC AUC: 0.713
Деревьев: 70    	точность на обучающем наборе: 0.905    	accuracy: 0.794    	ROC AUC: 0.718
Деревьев: 80    	точность на обучающем наборе: 0.902    	accuracy: 0.791    	ROC AUC: 0.715
Деревьев: 90    	точность на обучающем наборе: 0.903    	accuracy: 0.791    	ROC AUC: 0.713
Деревьев: 100    	точность на обучающем наборе: 0.903    	accuracy: 0.794    	ROC AUC: 0.715
Деревьев: 110    	точность на обучающем наборе: 0.903    	accuracy: 0.788    	R

Мы видим, что максимальная точность предсказаний у случайного леса, при количестве деревьев - 40 (мы зафиксировали глубину дерева из предыдущего исследования). Причем, надо заметить, что она очень не значительно выше, чем у дерева, при гораздо, большей сложности.

Итоговое дерево:

In [32]:
rfc = RandomForestClassifier(n_estimators=40, 
                             max_depth=13, 
                             random_state=9)
    
pred_rfc = rfc.fit(X_train, y_train).predict(X_valid)
    
print(f'точность на обучающем наборе: {round(rfc.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_rfc),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_rfc), 3)}')

точность на обучающем наборе: 0.913    	accuracy: 0.794    	ROC AUC: 0.72


***

#### Градиентный бустинг деревьев решений

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

Посмотрим как количество деревьев, влияет на точность градиентного бустинга

In [78]:
for n_estimators in range(10, 100, 5):
    
    gbc = GradientBoostingClassifier(n_estimators=n_estimators, 
                                     learning_rate=0.1, 
                                     max_depth=6,
                                     random_state=9)
    
    pred_gbc = gbc.fit(X_train, y_train).predict(X_valid)
    
    print(f'Деревьев: {n_estimators}\
    \tточность на обучающем наборе: {round(gbc.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_gbc),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_gbc), 3)}')

Деревьев: 10    	точность на обучающем наборе: 0.853    	accuracy: 0.769    	ROC AUC: 0.667
Деревьев: 15    	точность на обучающем наборе: 0.872    	accuracy: 0.773    	ROC AUC: 0.679
Деревьев: 20    	точность на обучающем наборе: 0.881    	accuracy: 0.782    	ROC AUC: 0.696
Деревьев: 25    	точность на обучающем наборе: 0.888    	accuracy: 0.791    	ROC AUC: 0.713
Деревьев: 30    	точность на обучающем наборе: 0.892    	accuracy: 0.782    	ROC AUC: 0.704
Деревьев: 35    	точность на обучающем наборе: 0.897    	accuracy: 0.779    	ROC AUC: 0.699
Деревьев: 40    	точность на обучающем наборе: 0.901    	accuracy: 0.782    	ROC AUC: 0.704
Деревьев: 45    	точность на обучающем наборе: 0.903    	accuracy: 0.785    	ROC AUC: 0.708
Деревьев: 50    	точность на обучающем наборе: 0.906    	accuracy: 0.785    	ROC AUC: 0.708
Деревьев: 55    	точность на обучающем наборе: 0.911    	accuracy: 0.782    	ROC AUC: 0.706
Деревьев: 60    	точность на обучающем наборе: 0.914    	accuracy: 0.785    	ROC

Видим, что при количестве деревьев 25 и их глубине всего лишь 6, мы получаем наилучший результат из всех предыдущих вариантов.
Запомним этот сет

In [79]:
gbc = GradientBoostingClassifier(n_estimators=25, 
                                     learning_rate=0.1, 
                                     max_depth=6,
                                     random_state=9)
    
pred_gbc = gbc.fit(X_train, y_train).predict(X_valid)
    
print(f'точность на обучающем наборе: {round(gbc.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_gbc),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_gbc), 3)}')

точность на обучающем наборе: 0.888    	accuracy: 0.791    	ROC AUC: 0.713


После смены разбивки у нас лес победил)

***

#### Простейшая нейронная сеть

Рассмотрим, как с данной задачей справится простая нейронная сеть

In [53]:
from sklearn.neural_network import MLPClassifier

In [54]:
mlp = MLPClassifier(random_state=9, solver='lbfgs')
    
pred_mlp = mlp.fit(X_train, y_train).predict(X_valid)
    
print(f'точность на обучающем наборе: {round(mlp.score(X_train, y_train), 3)}\
    \taccuracy: {round(accuracy_score(y_valid.values, pred_mlp),3)}\
    \tROC AUC: {round(roc_auc_score(y_valid.values, pred_mlp), 3)}')

точность на обучающем наборе: 0.706    	accuracy: 0.676    	ROC AUC: 0.5


Видим, что простая нейронная сеть не показывает выдающихся результатов на данном наборе данных

***

### Вывод:

Проведя анализ разных моделей классификации, мы пришли к выводу, что на данном наборе данных, самой эффективной на обучающем и валидационным наборах данных, проявил себя случайный лес деревьев решений. Применим его для тестового набора.

***

## 4. Проверим модель на тестовой выборке

Проверим случайный лес решающих деревьев с параметрами из предыдущего раздела со следующими параметрами:
 * Глубина дерева - 13
 * Количество деревьев - 40

In [82]:
rfc = RandomForestClassifier(n_estimators=40, 
                             max_depth=13, 
                             random_state=9)
    
pred_rfc = rfc.fit(X_train, y_train).predict(X_test)
    

print("Точность на обучающем наборе: {:.3f}".format(rfc.score(X_train, y_train)))
print("Точность на теством наборе: {:.3f}".format(rfc.score(X_test, y_test)))
print("ROC AUC:",  roc_auc_score(y_test, pred_rfc))

Точность на обучающем наборе: 0.913
Точность на теством наборе: 0.755
ROC AUC: 0.7035147392290251


Видим, что сслучайный лес на тестовых данных дает соизмеримый уровень точности. Модель удовлетворяет требованиям, можно тестировать на продакшене.